In [6]:
import json
import time
import numpy as np

In [4]:
f = open("../files/formations.json")
d = json.load(f)

In [99]:
n_pos = len(d["formations"][0]["positions_list"])

In [ ]:
connection_matrix = np.zeros((n_pos,n_pos), dtype=int)
for i, pos in enumerate(d["formations"][0]["positions_list"]):
    connection_matrix[i, pos["connections_list"]] = 1

In [103]:
nation_vector = np.array([1, 1, 1, 0, 0, 0, 0, 2, 2, 2, 2])
league_vector = np.array([1, 1, 1, 3, 3, 3, 3, 3, 3, 0, 0])
club_vector = np.array([1, 1, 1, 0, 0, 0, 4, 4, 4, 4, 4])
lc_vector = club_vector + 100 * league_vector

In [106]:
value_matrix = np.zeros((n_pos, n_pos), dtype=int)
for i in range(n_pos):
    if nation_vector[i] != 0:
        value_matrix[i, :] += np.equal(nation_vector[i], nation_vector)
    if league_vector[i] != 0:
        value_matrix[i, :] += np.equal(league_vector[i], league_vector)
        if club_vector[i] != 0:
            value_matrix[i, :] += np.equal(lc_vector[i], lc_vector)

In [107]:
link_matrix = np.multiply(connection_matrix, value_matrix)